In [18]:
%load_ext autoreload
%autoreload 2
%env WANDB_SILENT=true

from pathlib import Path
from copy import deepcopy
import matplotlib.pyplot as plt
import pandas as pd
import quantstats as qs
import os
import logging
import sys
import inspect


# Filter logs to only print from `nbtools`
class NbFilter(logging.Filter):
    def filter(self, record):
        return "nbtools" in record.getName()

qs.extend_pandas()
plt.style.use("seaborn-whitegrid")
pd.set_option('display.max_rows', 200)

if "freqtrade" not in os.listdir():
    import nest_asyncio
    nest_asyncio.apply()
    os.chdir("..")
    logger = logging.getLogger("freqtrade.nbtools")
    handler = logging.StreamHandler(stream=sys.stdout)
    handler.setFormatter(logging.Formatter("%(name)s - %(message)s"))
    logger.addHandler(handler)
    # logger.addFilter(NbFilter())
    logger.setLevel(logging.INFO)

print("Working Directory:", os.getcwd())

# Freqtrade Imports
from freqtrade.optimize.optimize_reports import text_table_add_metrics
from freqtrade.nbtools.preset import LocalPreset
from freqtrade.nbtools.backtest import backtest
from freqtrade.nbtools import plotting

# Constants
PATH_MOUNT = Path.cwd().parent / "mount"

# Custom function
def string_templating(string: str, data: dict, strip_doublequotes=False):
    _string = deepcopy(string)
    """Replace [|key|] in string with data["key"] """
    for key, value in data.items():
        to_replace = "[|{}|]".format(key)
        if strip_doublequotes:
            to_replace = "\"[|{}|]\"".format(key)
        if to_replace not in string:
            raise Exception(f"Not found formatting with key '{key}'")
        _string = _string.replace(to_replace, str(value))
    return _string

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: WANDB_SILENT=true
Working Directory: C:\CS\Python\crypto-collection\freqtrade\superft


In [13]:
def strategy_func():
    """ Start Strategy Code """
    from freqtrade.nbtools.strategy import INbStrategy
    from freqtrade.nbtools.remote_utils import load_pickle_asset
    from freqtrade.ml.lightning import TradingLightningModule
    from numpy.lib.npyio import save
    from numpy.lib.utils import info
    from pandas import DataFrame
    import numpy as np  # noqa
    import pandas as pd  # noqa
    import talib.abstract as ta
    
    MODULE_NAME = "15n30-randomforest"
    
    class NotebookStrategy(INbStrategy):
        # These attributes are REQUIRED!
        timeframe = "15m"
        minimal_roi = {"0": 99}
        stoploss = -0.99
        startup_candle_count: int = 500
            
        lightning_module: TradingLightningModule = None
        
        def populate_indicators(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
            if self.lightning_module is None:
                lightning_module = load_pickle_asset(MODULE_NAME, MODULE_NAME, version="[|version|]")
            
            dataframe["ema_10"] = ta.EMA(dataframe["close"], timeperiod=10)
            dataframe["ema_20"] = ta.EMA(dataframe["close"], timeperiod=20)
            return dataframe

        def populate_buy_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
            dataframe.loc[
                (
                    (dataframe['ema_10'] > dataframe['ema_20']) &
                    (dataframe['volume'] > 0)  # Make sure Volume is not 0
                ),
                'buy'] = 1
            return dataframe

        def populate_sell_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
            dataframe.loc[
                (
                    (dataframe['ema_10'] < dataframe['ema_20']) &
                    (dataframe['volume'] > 0)  # Make sure Volume is not 0
                ),
                'sell'] = 1
            return dataframe
    """ End Strategy Code """

In [19]:
data = {"version": 7}
sample_string = inspect.getsource(strategy_func)
print(string_templating(sample_string, data, strip_doublequotes=True))

def strategy_func():
    """ Start Strategy Code """
    from freqtrade.nbtools.strategy import INbStrategy
    from freqtrade.nbtools.remote_utils import load_pickle_asset
    from freqtrade.ml.lightning import TradingLightningModule
    from numpy.lib.npyio import save
    from numpy.lib.utils import info
    from pandas import DataFrame
    import numpy as np  # noqa
    import pandas as pd  # noqa
    import talib.abstract as ta
    
    MODULE_NAME = "15n30-randomforest"
    
    class NotebookStrategy(INbStrategy):
        # These attributes are REQUIRED!
        timeframe = "15m"
        minimal_roi = {"0": 0.02, "30": 0.01}
        stoploss = -0.01 
        startup_candle_count: int = 100
            
        lightning_module: TradingLightningModule = None
        
        def populate_indicators(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
            if self.lightning_module is None:
                lightning_module = load_pickle_asset(MODULE_NAME, MODULE_NAME, v